In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [3]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


In [4]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [5]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [6]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of ages:', feature_batch['age'])
    print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([38 61 64 60 66], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([1 1 0 0 1], shape=(5,), dtype=int32)


In [7]:
example_batch = next(iter(train_ds))[0]
example_batch

{'age': <tf.Tensor: id=90, shape=(5,), dtype=int32, numpy=array([38, 61, 64, 60, 66])>,
 'sex': <tf.Tensor: id=98, shape=(5,), dtype=int32, numpy=array([1, 1, 0, 0, 1])>,
 'cp': <tf.Tensor: id=93, shape=(5,), dtype=int32, numpy=array([1, 4, 4, 3, 4])>,
 'trestbps': <tf.Tensor: id=102, shape=(5,), dtype=int32, numpy=array([120, 148, 130, 120, 112])>,
 'chol': <tf.Tensor: id=92, shape=(5,), dtype=int32, numpy=array([231, 203, 303, 178, 212])>,
 'fbs': <tf.Tensor: id=95, shape=(5,), dtype=int32, numpy=array([0, 0, 0, 1, 0])>,
 'restecg': <tf.Tensor: id=97, shape=(5,), dtype=int32, numpy=array([0, 0, 0, 0, 2])>,
 'thalach': <tf.Tensor: id=101, shape=(5,), dtype=int32, numpy=array([182, 161, 122,  96, 132])>,
 'exang': <tf.Tensor: id=94, shape=(5,), dtype=int32, numpy=array([1, 0, 0, 0, 1])>,
 'oldpeak': <tf.Tensor: id=96, shape=(5,), dtype=float32, numpy=array([3.8, 0. , 2. , 0. , 0.1], dtype=float32)>,
 'slope': <tf.Tensor: id=99, shape=(5,), dtype=int32, numpy=array([2, 1, 2, 1, 1])>,
 '

In [8]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())
    
age = feature_column.numeric_column("age")
demo(age)

[[38.]
 [61.]
 [64.]
 [60.]
 [66.]]


In [9]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [10]:
thal = feature_column.categorical_column_with_vocabulary_list('thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

W0524 21:19:27.496213 15056 deprecation.py:323] From c:\users\j\documents\github\tutorials\tf_2_tutorials\venv\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:2655: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0524 21:19:27.499233 15056 deprecation.py:323] From c:\users\j\documents\github\tutorials\tf_2_tutorials\venv\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:4207: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0524 21:19:27.499233 15056 deprecation.py:323] From c:\users\j\documents\github\tutorials\tf_2_tutorials\venv\lib\site-package

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [11]:
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[-0.17470598  0.20210259 -0.04227317 -0.21672206 -0.24674118 -0.28283125
   0.20918357  0.40577564]
 [-0.17470598  0.20210259 -0.04227317 -0.21672206 -0.24674118 -0.28283125
   0.20918357  0.40577564]
 [-0.1322492  -0.15297686 -0.2637692  -0.23098738 -0.37045962 -0.45631957
   0.43766603 -0.27147397]
 [-0.1322492  -0.15297686 -0.2637692  -0.23098738 -0.37045962 -0.45631957
   0.43766603 -0.27147397]
 [-0.1322492  -0.15297686 -0.2637692  -0.23098738 -0.37045962 -0.45631957
   0.43766603 -0.27147397]]


In [12]:
thal_hashed = feature_column.categorical_column_with_hash_bucket('thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

W0524 21:19:27.535204 15056 deprecation.py:323] From c:\users\j\documents\github\tutorials\tf_2_tutorials\venv\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:4262: HashedCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [26]:
feature_columns = []

# numeric cols
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
    feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
#age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
#feature_columns.append(age_buckets)

# indicator cols
#thal = feature_column.categorical_column_with_vocabulary_list('thal', ['fixed', 'normal', 'reversible'])
#thal_one_hot = feature_column.indicator_column(thal)
#feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

In [27]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [28]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [29]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5,
          verbose=2
         )

Epoch 1/5


FailedPreconditionError: Table already initialized.
	 [[{{node sequential_3/dense_features_9/thal_embedding/thal_lookup/hash_table/table_init/LookupTableImportV2}}]] [Op:__inference_keras_scratch_graph_6468]